## Affine camera calibration

In [54]:
"""CS231A Homework 1, Problem 2.

DATA FORMAT
In this problem, we provide and load the data for you. Recall that in the
original problem statement, there exists a grid of black squares on a white
background. We know how these black squares are setup, and thus can determine
the locations of specific points on the grid (namely the corners). We also have
images taken of the grid at a front image (where Z = 0) and a back image (where
Z = 150). The data we load for you consists of three parts: real_XY,
front_image, and back_image. For a corner (0,0), we may see it at the (137, 44)
pixel in the front image and the (148, 22) pixel in the back image. Thus, one
row of real_XY will contain the numpy array [0, 0], corresponding to the real
XY location (0, 0). The matching row in front_image will contain [137, 44] and
the matching row in back_image will contain [148, 22].
"""

import numpy as np

def extend_real_coordinates(coords, z):        
    z_extend = z * np.ones((real_XY.shape[0], 1))
    hg_extend = np.ones((real_XY.shape[0], 1))
    zeros = np.zeros((real_XY.shape[0], 4))
    coords_x = np.concatenate((coords, z_extend, hg_extend, zeros), axis=1)
    coords_y = np.concatenate((zeros, coords, z_extend, hg_extend), axis=1)
    return np.concatenate((coords_x, coords_y), axis=0)


def compute_camera_matrix(real_XY, front_image, back_image):
    """Computes camera matrix given image and real-world coordinates.

    Args:
        real_XY: Each row corresponds to an actual point on the 2D plane.
        front_image: Each row is the pixel location in the front image (Z=0).
        back_image: Each row is the pixel location in the back image (Z=150).
    Returns:
        camera_matrix: The calibrated camera matrix (3x4 matrix).
    """
    
    real_XY_front = extend_real_coordinates(real_XY, 0)
    real_XY_back = extend_real_coordinates(real_XY, 150)
    real_XY_complete = np.concatenate((real_XY_front, real_XY_back), axis=0)
    
    image_XY_complete = np.concatenate(
        (front_image.flatten(order='F'), back_image.flatten(order='F')))
    
    lstsq_solution = np.linalg.lstsq(real_XY_complete, image_XY_complete)
    camera_matrix = np.concatenate(
        (np.reshape(lstsq_solution[0], (2, 4)), np.array([[0, 0, 0, 1]])), axis=0)
    
    return camera_matrix

def rms_error(camera_matrix, real_XY, front_image, back_image):
    """Computes RMS error of points reprojected into the images.

    Args:
        camera_matrix: The camera matrix of the calibrated camera.
        real_XY: Each row corresponds to an actual point on the 2D plane.
        front_image: Each row is the pixel location in the front image (Z=0).
        back_image: Each row is the pixel location in the back image (Z=150).
    Returns:
        rms_error: The root mean square error of reprojecting the points back
            into the images.
    """
    real_XY_front = extend_real_coordinates(real_XY, 0)
    real_XY_back = extend_real_coordinates(real_XY, 150)
    real_XY_complete = np.concatenate((real_XY_front, real_XY_back), axis=0)
    
    image_XY_complete = np.concatenate(
        (front_image.flatten(order='F'), back_image.flatten(order='F')))
    
    camera_params = np.concatenate((camera_matrix[0, :], camera_matrix[1, :]), axis=0)
    print(camera_params)
    calculated_coords = np.dot(real_XY_complete, camera_params)
    diff = calculated_coords - image_XY_complete
    rms = np.dot(diff, diff) / diff.shape[0]
    return rms
    

# Load the example coordinates setup.
real_XY = np.load('real_XY.npy')
front_image = np.load('front_image.npy')
back_image = np.load('back_image.npy')

camera_matrix = compute_camera_matrix(real_XY, front_image, back_image)
rmse = rms_error(camera_matrix, real_XY, front_image, back_image)

print ("Camera Matrix: \n", camera_matrix)
print ("")
print ("RMS Error: ", rmse)



[  5.31276507e-01  -1.80886074e-02   1.20509667e-01   1.29720641e+02
   4.84975447e-02   5.36366401e-01  -1.02675222e-01   4.43879607e+01]
Camera Matrix: 
 [[  5.31276507e-01  -1.80886074e-02   1.20509667e-01   1.29720641e+02]
 [  4.84975447e-02   5.36366401e-01  -1.02675222e-01   4.43879607e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]

RMS Error:  0.493849514748
